In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [2]:
repo_list = pd.DataFrame()
repo_list_query = f"""
SELECT repo_id, repo_name, repo_path from repo
WHERE repo_name = 'BiG-CZ' OR repo_name = 'ADIOS' or repo_name = 'diana-hep' or repo_name = 'FingerPrint' or
      repo_name = 'ipop-project' or repo_name = 'open-source-dataturbine';
    """
repo_list = pd.read_sql_query(repo_list_query, con=engine)
print(repo_list)

   repo_id                repo_name                  repo_path
0    25498                   BiG-CZ                github.com/
1    25486                    ADIOS      github.com/ornladios/
2    25515                diana-hep                github.com/
3    25476              FingerPrint  github.com/rocksclusters/
4    25490             ipop-project                github.com/
5    25473  open-source-dataturbine        bitbucket.org/OSDT/


In [3]:
#from datetime import date
import datetime 

current = datetime.date.today()
today = "'" + str(current) + "'"
print(today)

first_current = current.replace(day=1)
last_month = first_current - datetime.timedelta(days=1)
end_date = "'" + str(last_month) + "'"
print(end_date)

print

start = last_month - datetime.timedelta(days=365)
year_ago = "'" + str(start) + "'"
print(year_ago)

'2021-02-11'
'2021-01-31'
'2020-02-01'


In [4]:
value = 25440
affil_query = f"""
    select distinct(count(cmt_commit_hash)), cmt_author_affiliation
    from commits
    where 
        repo_id = {value}
        AND cmt_author_timestamp >= {year_ago}
        AND cmt_author_timestamp <= {end_date}
    group by cmt_author_affiliation;
    """
affil = pd.read_sql_query(affil_query, con=engine)
print(affil)

Empty DataFrame
Columns: [count, cmt_author_affiliation]
Index: []


In [5]:
affil_query = f"""
    select distinct(count(cmt_commit_hash)), cmt_author_affiliation
    from commits
    where 
        cmt_author_timestamp >= {year_ago}
        AND cmt_author_timestamp <= {end_date}
    group by cmt_author_affiliation;
    """
affil = pd.read_sql_query(affil_query, con=engine)
print(affil)

    count cmt_author_affiliation
0  137662                   NULL


In [6]:
affil_query = f"""
    select distinct(count(cmt_commit_hash)), cmt_author_affiliation
    from commits
    where 
        cmt_author_timestamp >= {year_ago}
        AND cmt_author_timestamp <= {end_date}
        AND cmt_author_email LIKE '%%bitnami%%'
    group by cmt_author_affiliation;
    """
affil = pd.read_sql_query(affil_query, con=engine)
print(affil)

Empty DataFrame
Columns: [count, cmt_author_affiliation]
Index: []


In [7]:
affil_all_query = f"""
    select cmt_author_name, cmt_author_affiliation, cmt_author_email
    from commits
    group by cmt_author_name, cmt_author_affiliation, cmt_author_email;
    """
affil_all = pd.read_sql_query(affil_all_query, con=engine)
print(affil_all)

                      cmt_author_name cmt_author_affiliation  \
0                             knteran                   NULL   
1                                ryan                   NULL   
2                     Fernando Posada                   NULL   
3                      Jeroen Demeyer                   NULL   
4                        Kim Liegeois                   NULL   
...                               ...                    ...   
4313                Roman Katerinenko                   NULL   
4314                   James Overfelt                   NULL   
4315                           rouson                   NULL   
4316                  JonMatthewsFCAS                   NULL   
4317  Barnhart, Katherine (Katy) Ruth                   NULL   

                             cmt_author_email  
0                          knteran@sandia.gov  
1                              tanaka@isi.edu  
2                            eposada@sissa.it  
3                  jdemeyer@sardonis.UG

In [8]:
affil_by_person_query = f"""
    select cmt_author_name, cmt_author_affiliation, cmt_author_email
    from commits
    where cmt_author_email like '%%asuraci%%'
    group by cmt_author_name, cmt_author_affiliation, cmt_author_email;
    """
affil_by_person = pd.read_sql_query(affil_by_person_query, con=engine)
print(affil_by_person)

Empty DataFrame
Columns: [cmt_author_name, cmt_author_affiliation, cmt_author_email]
Index: []


In [9]:
affil_by_person_query = f"""
                    SELECT
                        DISTINCT(cmt_commit_hash),
                        CASE WHEN contributors.cntrb_canonical IS NOT NULL THEN contributors.cntrb_canonical ELSE cmt_author_email END AS cntrb_canonical,
                        CASE WHEN canonical_full_names.cntrb_full_name IS NOT NULL THEN canonical_full_names.cntrb_full_name ELSE cmt_author_name END AS canonical_full_name,
                        cmt_author_name, cmt_author_email, cmt_author_affiliation, repo_id, cmt_author_timestamp 
                    FROM commits 
                        LEFT OUTER JOIN contributors ON cntrb_email = cmt_author_email
                        LEFT OUTER JOIN (
                            SELECT cntrb_canonical, cntrb_full_name 
                            FROM contributors
                            WHERE cntrb_canonical = cntrb_email
                        ) canonical_full_names
                        ON canonical_full_names.cntrb_canonical = contributors.cntrb_canonical
                    WHERE
                        repo_id = {value}
                        AND cmt_author_name NOT LIKE 'snyk%%'
                        AND cmt_author_name NOT LIKE '%%bot'
                        AND cmt_author_name != 'Spring Operator'
                        AND cmt_author_name != 'Spring Buildmaster'
                         AND cmt_author_timestamp >= {year_ago}
                         AND cmt_author_timestamp <= {end_date}
                    ORDER BY
                        cmt_author_affiliation;
                    """
affil_by_person = pd.read_sql_query(affil_by_person_query, con=engine)
print(affil_by_person)
affil_group = affil_by_person.groupby(['cmt_author_affiliation', 'canonical_full_name'], as_index=False).sum()[['cmt_author_affiliation', 'canonical_full_name']]
#print(affil_group)


Empty DataFrame
Columns: [cmt_commit_hash, cntrb_canonical, canonical_full_name, cmt_author_name, cmt_author_email, cmt_author_affiliation, repo_id, cmt_author_timestamp]
Index: []


KeyError: "None of [Index(['cmt_author_affiliation', 'canonical_full_name'], dtype='object')] are in the [columns]"

In [10]:
authorDF = pd.DataFrame()
authorDF = affil_by_person.cmt_author_affiliation.value_counts()
authorDF = authorDF.reset_index()
authorDF.columns = ['name', 'commits']
print(authorDF.head(10))

Empty DataFrame
Columns: [name, commits]
Index: []


# Old Stuff

In [11]:
affil_query = f"""
SELECT
   COLUMN_NAME
FROM
   information_schema.COLUMNS
WHERE
   TABLE_NAME = 'contributors';
    """
affil = pd.read_sql_query(affil_query, con=engine)
print(affil)

               column_name
0                 cntrb_id
1         cntrb_created_at
2               cntrb_fake
3            cntrb_deleted
4               cntrb_long
5                cntrb_lat
6               gh_user_id
7     data_collection_date
8          cntrb_last_used
9              data_source
10      cntrb_country_code
11             cntrb_state
12              cntrb_city
13          cntrb_location
14         cntrb_canonical
15         cntrb_full_name
16                gh_login
17                  gh_url
18             gh_html_url
19              gh_node_id
20           gh_avatar_url
21          gh_gravatar_id
22        gh_followers_url
23        gh_following_url
24            gh_gists_url
25          gh_starred_url
26    gh_subscriptions_url
27    gh_organizations_url
28            gh_repos_url
29           gh_events_url
30  gh_received_events_url
31             cntrb_login
32             cntrb_email
33           cntrb_company
34                 gh_type
35              cntrb_type
3

In [12]:
affilx_query = f"""
SELECT
   COLUMN_NAME
FROM
   information_schema.COLUMNS
WHERE
   TABLE_NAME = 'contributors_aliases';
    """
affilx = pd.read_sql_query(affilx_query, con=engine)
print(affilx)

            column_name
0  data_collection_date
1            cntrb_a_id
2          cntrb_active
3   cntrb_last_modified
4              cntrb_id
5           data_source
6           tool_source
7       canonical_email
8           alias_email
9          tool_version


In [13]:
affilx2_query = f"""
SELECT
   canonical_email, alias_email
FROM
   contributors_aliases
;
    """
affilx2 = pd.read_sql_query(affilx2_query, con=engine)
print(affilx2)

                                 canonical_email  \
0                raymat@master.cl.vanderbilt.edu   
1                raymat@master.cl.vanderbilt.edu   
2                         p.shama@vanderbilt.edu   
3                           mattwthomp@gmail.com   
4                             daico007@gmail.com   
...                                          ...   
1387                        19100080@lums.edu.pk   
1388                       ashish.gehani@sri.com   
1389  38260180+shoaibCS@users.noreply.github.com   
1390                  umesh.timalsina@ioe.edu.np   
1391                justin@localhost.localdomain   

                                      alias_email  
0                  raymatsumoto@Macintosh-2.local  
1                  ray.a.matsumoto@vanderbilt.edu  
2     33552857+uppittu11@users.noreply.github.com  
3                    matt.thompson@vanderbilt.edu  
4      43968221+daico007@users.noreply.github.com  
...                                           ...  
1387   3826

In [14]:
affil2_query = f"""
SELECT
   ca_domain, ca_affiliation
FROM
   contributor_affiliations
;
    """
affil2 = pd.read_sql_query(affil2_query, con=engine)
print(affil2)

                       ca_domain    ca_affiliation
0                    goggins.com              None
1                    samsung.com           Samsung
2            linuxfoundation.org  Linux Foundation
3                        ibm.com               IBM
4                    walmart.com           Walmart
..                           ...               ...
703          dgarnier@pivotal.io            VMware
704               git@garnier.wf            VMware
705  matthew.heidemann@gmail.com            VMware
706               me@lurraca.com            VMware
707           lurraca@pivotal.io            VMware

[708 rows x 2 columns]


In [15]:
affil_by_person_query = f"""
    select distinct(count(cmt_commit_hash))
    from commits
    where cmt_author_email = 'soumyadeep2007@gmail.com';
    """
affil_by_person = pd.read_sql_query(affil_by_person_query, con=engine)
print(affil_by_person)

   count
0      0


In [16]:
affil_by_person_query = f"""
    select cmt_author_name, cmt_author_affiliation, cmt_author_email
    from commits
    where cmt_author_name like '%%rraca%%'
    group by cmt_author_name, cmt_author_affiliation, cmt_author_email;
    """
affil_by_person = pd.read_sql_query(affil_by_person_query, con=engine)
print(affil_by_person)

Empty DataFrame
Columns: [cmt_author_name, cmt_author_affiliation, cmt_author_email]
Index: []


In [20]:
affil3_query = f"""
SELECT
   cntrb_company, cntrb_email, cntrb_canonical, cntrb_full_name, cntrb_login
FROM
   contributors
GROUP BY
   cntrb_company, cntrb_email, cntrb_canonical, cntrb_full_name, cntrb_login
;
    """
affil3 = pd.read_sql_query(affil3_query, con=engine)
affil3[0:20]

,cntrb_company,cntrb_email,cntrb_canonical,cntrb_full_name,cntrb_login
0,None,None,None,None,None
1,None,None,None,None,versat
2,None,micheldemessieres@laptop.home,micheldemessieres@Michels-Mini.home,Michel de Messieres,None
3,None,teodor.nikolov22@gmail.com,None,None,teonnik
4,None,Jason.Sheldon@psu.edu,Jason.Sheldon@psu.edu,Jason Sheldon,None
5,Entos.AI,dgasmith@icloud.com,malorian@me.com,Daniel Smith,dgasmith
6,None,None,None,None,cem204
7,None,hwsmith@colorado.edu,hwsmith@colorado.edu,Hanson Smith,None
8,None,roed@mit.edu,roed@mit.edu,roed314,None
9,None,mdmosby@sandia.gov,mdmosby@sandia.gov,Matthew Mosby,None


In [22]:
affil4_query = f"""
SELECT
   cntrb_email, cntrb_type
FROM
   contributors
GROUP BY
   cntrb_email, cntrb_type
;
    """
affil4 = pd.read_sql_query(affil4_query, con=engine)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(affil4)

                                            cntrb_email cntrb_type
0                                  craigcitro@gmail.com       None
1                               katy.barnhart@gmail.com       None
2                               stefanvanzwam@gmail.com       None
3                                     tomasbrod@azet.sk       None
4                                georgakoudis1@llnl.gov       None
5                                                  None       None
6                                   lefebvrejp@ornl.gov       None
7                           loftuser@LOFT-MacBook.local       None
8                                bangerth@math.tamu.edu       None
9                                   fxcoudert@gmail.com       None
10                       mschellenbergercosta@gmail.com       None
11                                        btovar@nd.edu       None
12                               myoga.murase@gmail.com       None
13                             bill.hoffman@kitware.com       

In [24]:
affil5_query = f"""
SELECT
   cntrb_email
FROM
   contributors
GROUP BY
   cntrb_email
;
    """
affil5 = pd.read_sql_query(affil5_query, con=engine)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(affil5)

                                            cntrb_email
0                                 timo.strunk@gmail.com
1                                               demunso
2                            berlinhendrik@netscape.net
3                   jychoi-hpc@users.noreply.github.com
4                                      kskedL@gmail.com
5                                          chchu@fb.com
6                                      shumow@gmail.com
7                                 daniel.paukner@gmx.de
8                                           pernet@john
9                                                   jya
10                                   opaco450@gmail.com
11           41344266+crockatt@users.noreply.github.com
12                              matthieu.dien@gmail.com
13                                 tbjuedeman@gmail.com
14                                     bmelosh@usgs.gov
15                                     gschwing@uno.edu
16                                     elove@san